In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn; seaborn.set()
from sklearn.metrics import mean_squared_error
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

**Loading  Data and study the rows and column values**

In [ ]:
data = pd.read_csv("../input/bitcoin_price_Training - Training.csv")
print(data.head(5))
print(data.tail(5))
data.dtypes
data.info()
data.describe()

Date is to be in Date-time format inorder to perform a time series study. In the current dataset the date is stated as an object which  is to be converted by parsing through the read-csv command  

In [ ]:
data = pd.read_csv("../input/bitcoin_price_Training - Training.csv",index_col= 'Date')
print(data.head(5)) 
data.info()
data.index = pd.to_datetime(data.index)
print(data.index)
data.head(5)

Now sort the dataset from oldest - recent 

In [ ]:
data = data.sort_index()
data.head()

In [ ]:
data['Close'].plot()
plt.ylabel("DAily Bitcoin price")

In this model I am trying to predict the Closing price of Bitcoin, and so I create a new object eliminating other columns

In [ ]:
data = data['Close']

In [ ]:
weekly = data.resample('W').sum()
weekly.plot()
plt.ylabel('Weekly bitcoin price')


In [ ]:
by_year = data.groupby(data.index.year).mean()
by_year.plot()

In [ ]:
by_weekday = data.groupby(data.index.dayofweek).sum()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
by_weekday.plot()

In [ ]:
by_weekday = data.groupby(data.index.dayofweek).mean()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
by_weekday.plot()

In [ ]:
by_day = data.groupby(data.index.dayofyear).mean()
by_day.plot()

In [ ]:
by_month = data.groupby(data.index.month).mean()
by_month.plot()

In [ ]:
by_quarter = data.groupby(data.index.quarter).mean()
by_quarter.plot()

In [ ]:
by_quarter = data.groupby(data.index.quarter)
by_quarter.plot()

In [ ]:
by_quarter_overall = data.groupby(data.index.quarter).mean()
by_quarter_overall.plot()

In [ ]:
weekend = np.where(data.index.weekday < 5, 'Weekday', 'Weekend')
by_time = data.groupby([weekend, data.index.year]).mean()
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
by_time.loc['Weekday'].plot(ax=ax[0], title='Weekdays')
by_time.loc['Weekend'].plot(ax=ax[1], title='Weekends')

In [ ]:
ts = data

In [ ]:
plt.plot(ts)

This part is inspired by: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/ 

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_for_stationary(input_data):
    r_mean = input_data.rolling(window = 7,center=False).mean()
    r_std = input_data.rolling(window = 7,center=False).std()
    
    # plotting the data
    given = plt.plot(input_data, color = 'blue', label = 'given_series')
    rolling_mean = plt.plot(r_mean, color = 'red', label = 'rolling_mean')
    rolling_std = plt.plot(r_std, color ='green', label = 'rolling_std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
     #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(input_data)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
      
test_for_stationary(ts)   

### The test statistic is very large compared to the crtical value. The above series show that rolling mean value varies  with time and is not stationary. The series needs to be Stationarized. we need to eliminate the trend and seasonality from the series to make it stationary


# Transforming the data - (to eliminate trend)

In [ ]:
ts_logtransformed = np.log(ts)
plt.plot(ts_logtransformed)

In [ ]:
ts_logtransformed.head(10)

# Smoothing by Rolling average

In [ ]:
Rolling_average = ts_logtransformed.rolling(window = 7, center= False).mean()
plt.plot(ts_logtransformed, label = 'Log Transformed')
plt.plot(Rolling_average, color = 'red', label = 'Rolling Average')
plt.legend(loc = 'best')


The red line shows the rolling mean. Note that since we are taking average of last 7 values, rolling mean is not defined for first 7 values. This can be observed as:

In [ ]:
Rolling_average.head(10)

 Lets subtract this Rolling average this from the original log transfomred series.

In [ ]:
log_Rolling_difference = ts_logtransformed - Rolling_average
log_Rolling_difference.head(10)
log_Rolling_difference.tail(10)

 Replace NAN with 0 in the above dataframe to avoid any errors in the future. 

In [ ]:
log_Rolling_difference.dropna(inplace=True)
plt.plot(log_Rolling_difference)

In [ ]:
test_for_stationary(log_Rolling_difference)

Use exponential_weighted_moving_average instead of moving average to improve the solution

In [ ]:
expwighted_avg = ts_logtransformed.ewm(halflife=7,min_periods=0,adjust=True,ignore_na=False).mean()
plt.plot(ts_logtransformed, label = 'Log transfomed')
plt.plot(expwighted_avg, color='red', label = 'exponential weighted average')
plt.legend(loc = 'best')

In [ ]:
expwighted_avg.head(10)

In [ ]:
log_expmovwt_diff = ts_logtransformed - expwighted_avg

test_for_stationary(log_expmovwt_diff)

Now the current time series is stationary,the TS (test_statistic) value is less than 1% critcal value. Since weights are assigned to values from the starting index. Therefore the test works  

Few other methods to eliminate Trend and seasonality.
Here i am implementing Differencing and Decompostion


In [ ]:
ts_logtransformed.plot()

## Seasonality Adjustment by Differencing
Seasonal components in a Time series can be elimnated by using differencing. If there is a seasonal component at level of one month, then it can be removed on an observation today by substracting the value from last month (Ex: Value(Oct 1)-Value(sep 1), value (oct 2 - Value(Sep 2),.....) We can substract last months data to the presents (giving a gap of 30 days) and first months data would not be available for modeling

I am trying to difference different types of seasonality and test for statioanarized data

In [ ]:
#X = ts_logtransformed
#diff = list()
#days_in_quarter = 91
#for i in range(days_in_quarter, len(X)):
 #   value = X[i] - X[i - days_in_quarter]
#    diff.append(value)
#plt.plot(diff)
#diff = pd.Series(diff)
#diff
#plt.plot(diff)

In [ ]:
ts_diff_logtrans = ts_logtransformed -ts_logtransformed.shift(7)
plt.plot(ts_diff_logtrans)
ts_diff_logtrans.head(10)

In [ ]:
ts_diff_logtrans.dropna(inplace=True)
test_for_stationary(ts_diff_logtrans)

The Dickey-Fuller test statis is very much less than 1% criticasl value than in the previous case. We can say that the Time series is stationary with 99% confidence 

## Decomposing

In [ ]:
decomposition = seasonal_decompose(ts_logtransformed)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts_logtransformed, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
decomposed_TS = residual
decomposed_TS.dropna(inplace=True)
test_for_stationary(decomposed_TS)


# Forecasting

In [ ]:
#ACF and PACF plots:
lag_acf = acf(ts_diff_logtrans, nlags=30)
lag_pacf = pacf(ts_diff_logtrans, nlags=50, method='ols')

In [ ]:
#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff_logtrans)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff_logtrans)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

In [ ]:
#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff_logtrans)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff_logtrans)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
pyplot.figure()
pyplot.subplot(211)
plot_acf(ts_diff_logtrans, ax=pyplot.gca(),lags=40)
pyplot.subplot(212)
plot_pacf(ts_diff_logtrans, ax=pyplot.gca(), lags=50)
pyplot.show()

Below are some observations from the plots.

In this plot, the two dotted lines on either sides of 0 are the confidence interevals. These can be used to determine the ‘p’ and ‘q’ values as:
the Lag values are identified from PACF and ACF plots for Autoregression(AR) and Moving Average(MA) parameters, p and q respectively
p – The lag value where the PACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case p=2, The PACF shows a significant lag on day 2 

q – The lag value where the ACF chart crosses the upper confidence interval for the first time. If you notice closely, in this case q=18, since the ACF shows a significant lag for 18th day.

The PACF  perhaps has  some significant lag at 7th, 8th, 12th, 15th day and so on, which also suggests that there is still some seasonality present in the differenced data. We will try different orders of ARIMA models to identify the best order by deriving the residual sum of squares(RSS) for each model. Lower RSS are considered to be the best ones

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
ts_diff_logtrans = ts_diff_logtrans.fillna(0)

# AR Model

In [ ]:
#model = ARIMA(ts_logtransformed, order=(20, 1, 0))  
#results_AR = model.fit(disp=-1)  
#plt.plot(ts_diff_logtrans)
#plt.plot(results_AR.fittedvalues, color='red', label = 'order 20')
#RSS = results_AR.fittedvalues-ts_diff_logtrans
#RSS.dropna(inplace=True)
#plt.title('RSS: %.4f'% sum(RSS**2))
#plt.legend(loc = 'best')

In [ ]:
#model = ARIMA(ts_logtransformed, order=(15, 1, 0))  
#results_AR = model.fit(disp=-1)  
#plt.plot(ts_diff_logtrans)
#plt.plot(results_AR.fittedvalues, color='red', label = 'order 15')
#RSS = results_AR.fittedvalues-ts_diff_logtrans
#RSS.dropna(inplace=True)
#plt.title('RSS: %.4f'% sum(RSS**2))
#plt.legend(loc = 'best')

In [ ]:
#model = ARIMA(ts_logtransformed, order=(12, 1, 0))  
#results_AR = model.fit(disp=-1)  
#plt.plot(ts_diff_logtrans)
#plt.plot(results_AR.fittedvalues, color='red', label = 'order 12')
#RSS = results_AR.fittedvalues-ts_diff_logtrans
#RSS.dropna(inplace=True)
#plt.title('RSS: %.4f'% sum(RSS**2))
#plt.legend(loc = 'best')

In [ ]:
#model = ARIMA(ts_logtransformed, order=(7, 1, 0))  
#results_AR = model.fit(disp=-1)  
#plt.plot(ts_diff_logtrans)
#plt.plot(results_AR.fittedvalues, color='red', label = 'order 7')
#RSS = results_AR.fittedvalues-ts_diff_logtrans
#RSS.dropna(inplace=True)
#plt.title('RSS: %.4f'% sum(RSS**2))
#plt.legend(loc = 'best')

In [ ]:
model = ARIMA(ts_logtransformed, order=(8, 1, 0))  
results_AR = model.fit(disp=-1)  
plt.plot(ts_diff_logtrans)
plt.plot(results_AR.fittedvalues, color='red', label = 'order 8')
RSS = results_AR.fittedvalues-ts_diff_logtrans
RSS.dropna(inplace=True)
plt.title('RSS: %.4f'% sum(RSS**2))
plt.legend(loc = 'best')

In [ ]:
model = ARIMA(ts_logtransformed, order=(2, 1, 0))  
results_AR = model.fit(disp=-1)  
plt.plot(ts_diff_logtrans)
plt.plot(results_AR.fittedvalues, color='red', label = 'order 2')
RSS = results_AR.fittedvalues-ts_diff_logtrans
RSS.dropna(inplace=True)
plt.title('RSS: %.4f'% sum(RSS**2))
plt.legend(loc ='best')



In [ ]:
print(results_AR.summary())

# MA model

In [ ]:
model = ARIMA(ts_logtransformed, order=(0, 1,18)) 
results_MA = model.fit(disp=-1)  
plt.plot(ts_diff_logtrans)
plt.plot(results_MA.fittedvalues, color='red')
RSS = results_MA.fittedvalues-ts_diff_logtrans
RSS.dropna(inplace=True)
plt.title('RSS: %.4f'% sum(RSS**2))


In [ ]:
#model summary
print(results_MA.summary())


In [ ]:
plt.plot(ts_logtransformed, label = 'log_tranfromed_data')
plt.plot(results_MA.resid, color ='green',label= 'Residuals')
plt.title('MA Model Residual plot')
plt.legend(loc = 'best')

In [ ]:
results_MA.resid.plot(kind='kde')
plt.title('Density plot of the residual error values')
print(results_MA.resid.describe())

# ARIMA Combined model

In [ ]:
model = ARIMA(ts_logtransformed, order=(8, 1, 18))  
results_ARIMA = model.fit(trend= 'nc', disp=-1)  
plt.plot(ts_diff_logtrans)
plt.plot(results_ARIMA.fittedvalues, color='red', label = 'p =8, q =18')
RSS =results_ARIMA.fittedvalues-ts_diff_logtrans
RSS.dropna(inplace=True)
plt.title('RSS: %.4f'% sum(RSS**2))
plt.legend(loc='best')

model = ARIMA(ts_logtransformed, order=(20, 1, 18)) 
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_diff_logtrans)
plt.plot(results_ARIMA.fittedvalues, color='red', label = 'order 15')
RSS = results_ARIMA.fittedvalues-ts_diff_logtrans
RSS.dropna(inplace=True)
plt.title('RSS: %.4f'% sum(RSS**2))
plt.legend(loc = 'best')

### Searching ARIMA parameters.
The above plots and models derive varying residual RSS. evaluating all these models might be difficult. SO we do a grid search on all possible ARIMA parameters. We look for different p,d,q combinations, and find the best combination.
p = 7 ,10,13,16,19
d = 0 to 2
taking 10 runs of the model and this will take some time to run

The following markdown is a python code that helps to evaluate different model performances to evaluate best ARIMA hyperparameters

import warnings
def evaluate_arima_model(data_set, arima_order):
    model = ARIMA(data_set, order=arima_order)
    results_ARIMA = model.fit(disp=-1) 
    RSS_diff = results_ARIMA.fittedvalues-ts_diff_logtrans
    RSS = RSS_diff**2
    return RSS
def evaluate_models(dataset, p_values, d_values):
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            order = (p,d,18)
            try:
                rss = evaluate_arima_model(dataset, order)
                if rss < best_score:
                    best_score, best_cfg = rss, order
                print('ARIMA%s RSS=%.3f' % (order,rss))
            except:
                continue
    print('Best ARIMA%s RSS=%.3f' % (best_cfg, best_score))
p_values = range(8,20,3)
d_values = range(0,3)
warnings.filterwarnings('ignore')
evaluate_models(ts_logtransformed,p_values,d_values)

We can derive the Summary of the ARIMA model as follows explaining the results

In [ ]:
#model summary
print(results_ARIMA.summary())

In [ ]:
plt.plot(ts_logtransformed, label = 'log_tranfromed_data')
plt.plot(results_ARIMA.resid, color ='green',label= 'Residuals')
plt.title('ARIMA Model Residual plot')
plt.legend(loc = 'best')

In [ ]:
results_ARIMA.resid.plot(kind='kde')
plt.title('Density plot of the residual error values')
print(results_ARIMA.resid.describe())

In [ ]:
test = pd.read_csv("../input/bitcoin_price_1week_Test - Test.csv",index_col= 'Date')
test.index = pd.to_datetime(test.index)
test = test['Close']
test = test.sort_index()
test

## Using Combined ARIMA model to predict and forecast

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum.head())

In [ ]:
predictions_ARIMA_log = pd.Series(ts_logtransformed.iloc[0], index=ts_logtransformed.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(data)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-data)**2)/len(data)))

In [ ]:
dates = [pd.Timestamp('2017-08-01'), pd.Timestamp('2017-08-02'), pd.Timestamp('2017-08-03'),pd.Timestamp('2017-08-04'), pd.Timestamp('2017-08-05'), pd.Timestamp('2017-08-06'), pd.Timestamp('2017-08-07')]

forecast = pd.Series(results_ARIMA.forecast(steps=7)[0],dates)
forecast = np.exp(forecast)
print(forecast)
error = mean_squared_error(test, forecast)
print('Test MSE: %.3f' % error)

In [ ]:
plt.plot(forecast, color ='green', label ='Predicted rates')
plt.plot(test, color = 'red', label = 'Observed from test data')
plt.title('RMSE: %.4f'% np.sqrt(sum((forecast-test)**2)/len(data)))
plt.legend(loc = 'best')

## Using MA model to predict and forecast

Here we can see that the AR and MA, and the combined ARIMA models have almost the same RSS but MA model is significantly better in computational performance. So we use it to predict the price.



In [ ]:
predictions_MA_diff = pd.Series(results_MA.fittedvalues, copy=True)
print(predictions_MA_diff.head())

In [ ]:
predictions_MA_diff_cumsum = predictions_MA_diff.cumsum()
print(predictions_MA_diff_cumsum.head())

Notice that the predictions arent starting from 2013-04-28. This is because we took a lag by 1 and first element doesn’t have anything before it to subtract from. The way to convert the differencing to log scale is to add these differences consecutively to the base number. An easy way to do it is to first determine the cumulative sum at index and then add it to the base number

In [ ]:
predictions_MA_log = pd.Series(ts_logtransformed.iloc[0], index=ts_logtransformed.index)
predictions_MA_log = predictions_MA_log.add(predictions_MA_diff_cumsum,fill_value=0)
predictions_MA_log.head()

In [ ]:
predictions_MA = np.exp(predictions_MA_log)
plt.plot(data)
plt.plot(predictions_MA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_MA-data)**2)/len(data)))

In [ ]:
dates = [pd.Timestamp('2017-08-01'), pd.Timestamp('2017-08-02'), pd.Timestamp('2017-08-03'),pd.Timestamp('2017-08-04'), pd.Timestamp('2017-08-05'), pd.Timestamp('2017-08-06'), pd.Timestamp('2017-08-07')]

forecast = pd.Series(results_MA.forecast(steps=7)[0],dates)
forecast = np.exp(forecast)
print(forecast)
error = mean_squared_error(test, forecast)
print('Test MSE: %.3f' % error)

In [ ]:
plt.plot(forecast, color ='green', label ='Predicted rates')
plt.plot(test, color = 'red', label = 'Observed from test data')
plt.title('RMSE: %.4f'% np.sqrt(sum((forecast-test)**2)/len(data)))
plt.legend(loc = 'best')

Much work has to be done on eliminating hte seasonality in the above time series. More methods licke curve fitting and differnce can be extended to identify seasonality and remove the seasonal component from the data.

An improved model may be to subtract the average coin price from the same calendar month in the previous year, rather than the same day.

We can start off by resampling the dataset to a monthly average price. The resampling methods handles the concept of leap years while eliminating the offset  (for example feb has only 28 days ). 

In [ ]:
monthly_mean = data.resample('M').mean()
monthly_mean
print(monthly_mean.head(13))
monthly_mean.plot()

## One more alternative to derive the forecasting performance

In [ ]:
test_logtransformed = np.log(test)

In [ ]:
history = [x for x in ts_logtransformed]
predictions = list()
for t in range(len(test)):
    output = results_MA.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test_logtransformed[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test_logtransformed, predictions)
print('Test MSE: %.3f' % error)


Much work has to be done on eliminating hte seasonality in the above time series. More methods licke curve fitting and differnce can be extended to identify seasonality and remove the seasonal component from the data.

An improved model may be to subtract the average coin price from the same calendar month in the previous year, rather than the same day.

We can start off by resampling the dataset to a monthly average price. The resampling methods handles the concept of leap years while eliminating the offset  (for example feb has only 28 days ). 

In [ ]:
from fbprophet import Prophet

In [ ]:
data.head()

In [ ]:
data_prophet = data.copy()
data_prophet = pd.DataFrame(data_prophet)
data_prophet.reset_index(drop=False, inplace=True)
data_prophet.columns =['ds','y']
data_prophet

In [ ]:
m = Prophet()
m.fit(data_prophet)
future = m.make_future_dataframe(periods=7, freq='D')
forecast = m.predict(future)
m.plot(forecast)
data.plot()


In [ ]:
m.plot_components(forecast)

In [ ]:
forecast.columns

In [ ]:
forecasted_values = forecast[['ds', 'yhat']].tail(7)

In [ ]:
forecasted_values = forecasted_values.set_index('ds')
forecasted_values.columns = ['y']
forecasted_values

In [ ]:
mean_squared_error(forecasted_values['y'],test)

## RNN

- In Progress

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

The Model needs further  improvement. Will have to work on few other approaches such as Facebooks Prophet approach and Keras Starter. Much information on theses approaches can be found on https://research.fb.com/prophet-forecasting-at-scale/ and https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
